# Crystals of Sound

The following notebook contains a Python implementation of the calculations needed to produce the results presented in the manuscript "Crystals of sound: applying the physics of phase transitions to musical intonation." A few precalculated data sets are provided to produce the results presented in the paper, or you can run the simulation to produce your own data; however, the section titled "Solving the Mean Field Model" can be quite slow depending on input paramters.

Where indicated, there are a few parameters which can be easily adjusted to alter the output of the calculation to explore other tuning systems. Lines of programming preceded by a `#` are comments, used to explain portions of the code, or simply remove certain lines from the program. For a more in depth discussion of the methods in this notebook, see [this article in Science Advances](https://www.science.org/doi/10.1126/sciadv.aav8490).

The following packages will need to be installed in order to run the code: `numpy`, `matplotlib`, `IPython`, `time`, `scipy.signal` and `scipy.fft`.

In [ ]:
import numpy as np
from scipy.fft import fft
from numpy.random import rand, normal
from scipy.signal import find_peaks
import pickle
from time import sleep
from IPython import display

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

## Computing the Dissonance Function

This computes the dissonance between two pure sine wave tones, plotted for a variety of critical bandwidth $\omega_c$ values.

In [ ]:
def Diss(fa, fb, wc):
    '''
    Given two frequencies fa, fb, computes the simple 
    two-tone dissonance for an interval x = log_2(fa/fb), 
    with critical bandwidth wc.
    '''
    dx = np.abs(np.log2(np.outer(fa,1/fb)))
    return np.exp(-np.log(dx/wc)**2)/wc

In [ ]:
wc_arr = np.array([0.03, 0.05, 0.1, 0.2]);

f_min = 1; ## unison
f_max = 2; ## one octave
f_pts = 500;
f_arr = np.linspace(f_min,f_max,f_pts);
dx_arr = 1200*np.abs(np.log2(np.outer(f_arr,1)));

fig_simple_diss = plt.figure(figsize=(10,6));
ax_simple_diss = fig_simple_diss.add_subplot(111);
ax_simple_diss.set_xlabel("Interval (cents)");
ax_simple_diss.set_ylabel("Dissonance (arb. units)");
ax_simple_diss.set_title("Simple 2-tone Dissonance function");

for wc in wc_arr:
    ax_simple_diss.plot(dx_arr, Diss(f_arr, 1, wc), label = f"wc={wc}");
    
ax_simple_diss.legend();

## Sawtooth Dissonance

To reproduce the plot in Figure 2, set `n_partials=10` and `wc=36/1200`. For Figure 13, set `n_partials=14` and `wc=12/1200`. A few 5-limit JI ratios are included for comparison, denoted in red, dashed vertical lines.

In [ ]:
## literal recreation of Matlab 'sawtooth_dissonance.m'
def sawtooth_diss(wc, n_partials, farr):
    partials = np.arange(n_partials)+1;
    amps = 1/partials;
    
    dtab = np.zeros(len(farr));
    for f in range(len(farr)):
        diss = 0;
        f2 = farr[f];
        for j in range(len(partials)):
            m = partials[j];
            dx = np.abs(np.log2(partials/m)-f2);
            diss += np.sum(np.minimum(amps[j],amps)**0.606 * np.exp(-np.log(dx/wc)**2))/wc;
        dtab[f] = diss;
    
    return dtab
    
## Set Partials and critical bandwidth here ########################
n_partials = 10; 
wc = 36/1200; 
####################################################################

f_min = -4; f_max = 4; 
f_pts = 8000;
f_arr = np.linspace(f_min, f_max, f_pts);
dtab = sawtooth_diss(wc, n_partials, f_arr);

fn = f_arr[np.logical_and(0<=f_arr,f_arr<1)]; ## frequency range from one octave
## add up dissonance from each octave
dn = np.sum(dtab.reshape( (len(fn), len(dtab)//len(fn)), order='F' ), axis=1); 

fig_full_diss = plt.figure(figsize=(10,6));
ax_full_diss = fig_full_diss.add_subplot(111);
ax_full_diss.set_xlabel("Interval (cents)");
ax_full_diss.set_ylabel("Dissonance (arb. units)");
ax_full_diss.set_title(f"Full Dissonance function, w_c={wc}, N={n_partials} harmonics");

ax_full_diss.plot(1200*fn, dn)

ratios = 1200*np.log2(np.array([9/8,6/5,5/4,4/3,3/2,8/5,5/3,16/9]));    ## defines JI ratios of interest
ratio_names = ['9/8','6/5','5/4','4/3','3/2','8/5','5/3','16/9'];

[ax_full_diss.axvline(r,ymax=0.95,color='red',ls='dashed',lw=2) for r in ratios];
[ax_full_diss.text(ratios[j],1*max(dn),ratio_names[j],fontsize=12, ha='center') for j in range(len(ratio_names))];


This plot indicates the Fourier components of the Dissonance function; the most negative component is an order parameter for the mean field model and is highlighted here in red. For the case of $\omega_c=36/1200$, the component with $k=12$ is most negative, suggesting a 12-fold ordering of pitches.

In [ ]:
fig_diss_fourier = plt.figure(figsize=(10,6));
ax_diss_fourier = fig_diss_fourier.add_subplot(111);
ax_diss_fourier.set_xlabel("k");
ax_diss_fourier.set_ylabel("D_k");
ax_diss_fourier.set_title("Fourier Components D_k of Dissonance function");

k_max = 60; 
k_arr = np.arange(len(fn))[1:k_max];
fourier_arr = 2/len(fn)*np.real(fft(dn))[1:k_max];

barlist = ax_diss_fourier.bar(k_arr, fourier_arr);
barlist[np.argmin(fourier_arr)].set_color('r');


## Solving the Mean Field Model

The code to solve the mean field model can be fairly slow, especially near the transition temperatures, where the probability distributions take longer to converge to the optimal values. This can be sped up by increasing the value of `maxerr` or decreasing the value of `maxiter`, at the cost of precision in the simulation. The option `show_progress` can be set to `True` or `False`; setting it to `True` will slow down the simulation, but allow you to watch as the solver converges to the optimal distribution at each temperature. For your convenience, a few pre-calculated data sets are provided, using both the fixed critical bandwidth and variable bandwidth dissonance functions.

In [ ]:
def SolveMeanField(p, temp, D_arr, max_err, max_iter, show_progress=False):
    err = 1; 
    count = 0;
    
    errs = []; Fs = [];
    
    p = np.abs(p+normal(size=p.shape)*1e-3);  ## add jitter to destabilize
    p /= (np.sum(p)/n_bins);                   ## renormalize
    
    if show_progress==True:
        ## initialize plots for progress
        fig_progress = plt.figure(figsize=(8,8));
        gs = fig_progress.add_gridspec(2,2);
        
        ax_prog_p = fig_progress.add_subplot(gs[0,:]);
        ax_prog_p.set_title(f"T = {temp}");
        ax_prog_p.set_xlabel("Frequency (cents)");
        ax_prog_p.set_ylabel("Probability Density");
        line_p, = ax_prog_p.plot(1200*f_arr, p);
        
        ax_prog_f = fig_progress.add_subplot(gs[1,0]);
        ax_prog_f.set_xlabel("Iteration");
        ax_prog_f.set_ylabel("Free Energy");
        line_f, = ax_prog_f.plot(np.nan);
        
        ax_prog_err = fig_progress.add_subplot(gs[1,1]);
        ax_prog_err.set_xlabel("Iteration");
        ax_prog_err.set_ylabel("Error (log scale)");
        ax_prog_err.set_yscale("log");
        line_err, = ax_prog_err.plot(err);

    if show_progress==False:
        display.clear_output(wait=True)
        print(f"T={np.round(temp,3)} \n ---------------- \n Iterations: <100");

    while err>max_err and count<max_iter:
        count+=1;
        p_arr = np.tile(p, (len(p),1) );
        
        intgl = np.sum(D_arr*p_arr,axis=1)/n_bins;
        num = np.exp(-intgl/temp);  ## numerator of Eqn. 4
        denom = np.sum(num)/n_bins;  ## denom of Eqn. 4 (partition function)
        
        E_tot = 0.5*np.sum(p*intgl)/n_bins;
        S_tot = -np.sum(p*np.log(p))/n_bins;
        F = E_tot - temp*S_tot;
        
        p_new = num/denom;
        err = np.sum((p_new-p)**2);  ## sum of squares difference
        p = (p+0.5*p_new);           ## iterate by interpolate between LHS and RHS of Eqn. 4
        p/=np.sum(p)/n_bins;          ## renormalize
        
        errs.append(err); Fs.append(F);
        
        if show_progress==True and count%10==0:
            display.clear_output(wait=True)
            
            line_p.set_ydata(p);
            ax_prog_p.set_ylim(0,1.1*max(p));
            
            x_axis = np.arange(len(Fs))+1
            line_f.set_xdata(x_axis);
            line_f.set_ydata(np.array(Fs));
            ax_prog_f.set_xlim(1,max(x_axis));
            ax_prog_f.set_ylim(0,max(Fs));
            
            line_err.set_xdata(x_axis);
            line_err.set_ydata(np.array(errs));
            ax_prog_err.set_xlim(1,max(x_axis));
            ax_prog_err.set_ylim(max_err,max(errs));
            
            display.display(plt.gcf())
            sleep(0.5);

        if show_progress==False and count%100==0:
            display.clear_output(wait=True)
            print(f"T={np.round(temp,3)} \n ---------------- \n Iterations: {count}");
        
    if show_progress==True:
        plt.close(fig_progress)
        
    return (E_tot, S_tot, p);

In [ ]:
n_bins = 1200;
f_arr = np.linspace(0,1,n_bins);
f1, f2 = np.meshgrid(f_arr,f_arr);
D_arr = np.interp(np.abs(f1-f2), fn, dn); ## 1D linear interpolation, could use a more sophisticated one if you like

## setup temperature range for sweep
t_min = 10; t_max = 30; t_pts = 200;
t_arr = np.linspace(t_max, t_min, t_pts);

## error threshold and max iterations for solver
max_err = 1e-10;
max_iter = 1e4;

Ds = np.zeros(len(t_arr));                ## store total dissonance at each T
Ss = np.zeros(len(t_arr));                ## store total entropy at each T
Ps = np.zeros((len(f_arr), len(t_arr)));  ## store prob dist at each T

## initialize random probability distribution; fix normalization
p = rand(n_bins);
p /= (np.sum(p)/n_bins);

## do first iteration; each subsequent iteration will use previous solution as starting point
Ds[0], Ss[0], Ps[:,0] = SolveMeanField(p, t_arr[0], D_arr, max_err, max_iter, show_progress=False);

for t in range(1,len(t_arr)):
    Ds[t], Ss[t], Ps[:,t] = SolveMeanField(Ps[:,t-1], t_arr[t], D_arr, max_err, max_iter, show_progress=False);
    

### Saving data

To save the data from a simulation run, uncomment and run the following code. Be careful to check the file name before saving to avoid overwriting data.

In [ ]:
##data = {'Temps':t_arr, 'Freqs':f_arr, 'Diss':Ds, 'Entropy':Ss, 'Probs':Ps}

##with open(f'MFProb_N={n_partials}_wc={wc}_err={max_err}_iter={max_iter}.pickle','wb') as file:
##    pickle.dump(data, file);

### Loading saved data
To use one of the provided data sets, first ensure the pickle file is saved in the same folder as this notebook. Remove the comment lines from the following cell and run the code; this should set each array with the values from the pickle file, allowing you to run the remainder of the notebook using that data.

To produce the results shown in Figures 3 and 6: `MFProb_N=10_wc=0.03_err=1e-12_iter=10000.0.pickle`

To produce the results shown in Figures 14 and 15: `MFProb_N=14_wc=0.01_err=1e-12_iter=10000.0.pickle`

In [ ]:
##with open(f'MFProb_N=10_wc=0.03_err=1e-12_iter=10000.0.pickle','rb') as file:
##    data = pickle.load(file);

##t_arr = data['Temps']; 
##f_arr = data['Freqs'];
##Ds = data['Diss'];
##Ss = data['Entropy'];
##Ps = data['Probs'];

This animation shows the evolution from disordered sound to ordered distributions as a function of temperature; you can change the speed of the animation by change the value of `frame_rate`.,

In [ ]:
mf_anim = plt.figure(figsize=(10,6));
ax_mf_anim = mf_anim.add_subplot(111);
ax_mf_anim.set_xlabel("Frequency (cents)");
ax_mf_anim.set_ylabel("Probability Density");
ax_mf_anim.set_xlim(0,1200);
ax_mf_anim.set_ylim(0,max(Ps[:,0]));
line_mf, = ax_mf_anim.plot(1200*f_arr, Ps[:,0], label = f"T={t_arr[0]}");
ax_mf_anim.legend(loc='upper right');

frame_rate = 20 ## frames per second (roughly)
for t in range(1,len(t_arr)):
    sleep(1/frame_rate);
    display.clear_output(wait=True);
    line_mf.set_ydata(Ps[:,t]);
    line_mf.set_label(f"T={t_arr[t]}");
    ax_mf_anim.set_ylim(0,max(Ps[:,t]));
    ax_mf_anim.legend(loc='upper right');
    display.display(plt.gcf());
    
plt.close(mf_anim);   

The following plot shows the previous animation in a still image, as shown in Figures 3 and 14.

In [ ]:
## Plot probabilities across range of T
t2, f2 = np.meshgrid(t_arr, f_arr);

## set color scale to make tunings visible
col_min=-15; col_max=2; col_pts=200;
levels = np.logspace(col_min,col_max,col_pts);
ticks = np.logspace(col_min,col_max,col_max-col_min+1);

fig_mf_prob = plt.figure(figsize=(8,8));
ax_mf_prob = fig_mf_prob.add_subplot(111);
cplot = ax_mf_prob.contourf(1200*f2, t2, Ps, cmap='inferno', norm=mpl.colors.LogNorm(), levels=levels, vmin=0.1);
cbar = fig_mf_prob.colorbar(cplot, ticks=ticks);
ax_mf_prob.set_xlabel("Frequency (cents)");
ax_mf_prob.set_ylabel("Temperature");
ax_mf_prob.set_title("Probability Density");

In this cell, select a temperature from the range shown above; this will then plot the pitch probability distribution nearest to that temperature. Peaks in the distribution have their positions indicated wtih dashed, green vertical lines; the height of what is considered a "peak" can be set with the line `height = 0.1`. Gridlines are included every 100 cents for comparison to 12-TET; a selection of 5-limit JI ratios are indicated in dotted, red vertical lines for comparison as well.

In [ ]:
## Pick a temperature ############################
T = 18;
##################################################

T_ind = np.argmin(np.abs(t_arr-T));
p_arr = np.roll(Ps[:,T_ind], -np.argmax(Ps[:,t]));

## find peaks in probability distribution
height = 0.1; ## set height of peak to look for
mf_inds = find_peaks(p_arr,height=height)[0];
mf_pitches = 1200*f_arr[mf_inds];

fig_peaks = plt.figure(figsize=(12,6));
ax_peaks = fig_peaks.add_subplot(111);
ax_peaks.plot(1200*f_arr, p_arr, label = f'T={T}');
ax_peaks.legend(loc='upper right');
ax_peaks.set_xlabel("Frequency (cents)");
ax_peaks.set_ylabel("Probability Density");
[ax_peaks.axvline(f,ymin=0.5,ymax=0.95,color='green',ls='dashed',lw=2) for f in mf_pitches];

## Compare to 12-TET
et_ticks = np.arange(0,1201,100);
et_names = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B','C'];
ax_peaks.grid(which='major', axis='x');
ax_peaks.set_xticks(et_ticks);
[ax_peaks.text(et_ticks[j],1.06*np.max(p_arr),et_names[j],fontsize=12, ha='center') for j in range(len(et_names))];

## Compare to 5-limit JI
ratios = 1200*np.log2(np.array([1, 16/15, 9/8, 6/5, 5/4, 4/3, 45/32, 64/45, 3/2, 8/5, 5/3, 16/9, 15/8]));    ## defines JI ratios of interest
ratio_names = ['1','16/15','9/8','6/5','5/4','4/3','45/32','64/45','3/2','8/5','5/3','16/9','15/8'];
[ax_peaks.axvline(r,ymin=0.08,ymax=0.5,color='red',ls='dotted',lw=2) for r in ratios];
# [ax_peaks.text(ratios[j],0.15,ratio_names[j],fontsize=12, ha='center') for j in range(len(ratio_names))];



## Plots for Physics Nerds

Below we plot the the free energy vs. temperature, as well as Fourier components of the mean field probability distributions and their derivatives, which serve as order parameters for the model. The peaks in the the derivative demonstrate the existence of phase transitions.

In [ ]:
## Plot Free energy vs. T
fig_freeE = plt.figure(figsize=(10,6));
ax_freeE = fig_freeE.add_subplot(111);
ax_freeE.plot(t_arr, Ds-t_arr*Ss, marker='.', ms=10);
ax_freeE.set_xlabel("Temperature");
ax_freeE.set_ylabel("Free Energy");

In [ ]:
pk_arr = np.abs(fft(Ps,axis=0).T)/len(f_arr)
k_max = 20;

fig_mf_fourier = plt.figure(figsize=(8,6));
ax_mf_fourier = fig_mf_fourier.add_subplot(211);
ax_mf_fourier.set_xlabel("Temperature");
ax_mf_fourier.set_ylabel(r"$|p_k|$");
ax_mf_dfdt = fig_mf_fourier.add_subplot(212);
ax_mf_dfdt.set_xlabel("Temperature");
ax_mf_dfdt.set_ylabel(r"$|\frac{d}{dT} p_k|$");

for k in range(k_max):
    ax_mf_fourier.plot(t_arr, pk_arr[:,k]);
    
    dt = np.diff(t_arr);
    dpk = np.diff(pk_arr[:,k]);
    ax_mf_dfdt.plot(t_arr[:-1]+dt[0], np.abs(dpk/dt));
    